## **Exercício para o Portifólio de vocês**

1. Abra um colab novo 
2. Baixe o mesmo conjunto de textos que usamos nesta aula
3. Selecione 10 classes do seu interesse 
4. Treine dois modelos, faça tuning e separe por abordagens 

**A entrega será somente dia 24/04. E vale 3 pontos** 

**Pensem que alguém vai ler este código depois de vocês fazerem**

Critérios de avaliação: 
- Não vou me apegar a assertividade do modelo 
- O critério será a organização e conclusões ao longo do codigo.

In [ ]:
# Quando usamos "as", colocamos algum alias ,(atribuir um "Apelido"), a uma determinada biblioteca.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
import warnings

from sklearn.datasets import fetch_20newsgroups # dataset de texto para classificação contendo 20 classes
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer # carrega Vectorizer e TFIDF
from sklearn.naive_bayes import MultinomialNB # algoritmo do Naive Bayes
from sklearn.pipeline import Pipeline # Cria pipeline contendo todas as transformações e modelo
from nltk.stem.snowball import SnowballStemmer # Função que retorna a palavra a sua raiz
from sklearn.linear_model import SGDClassifier # Algoritmo Gradient Descendente Stocastico
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

warnings.simplefilter('ignore')

#### **Carrega Dataset**

In [ ]:
newsgroups = fetch_20newsgroups(subset='train') # Carrega o dataset de treinamento do fetch 20 news groups

In [ ]:
list(newsgroups.target_names) # retorna as classes disponíveis para treinamento

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian','talk.religion.misc','rec.motorcycles','rec.sport.baseball',
              'rec.sport.hockey','comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware'] # Lista com as classes que vamos trabalhar
df_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
df_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True)

In [ ]:
df_train.target_names # Apresenta as classes

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.religion.misc']

#### **Feature Enginnering** 

In [ ]:
# Converta uma coleção de documentos de texto em uma matriz de contagem de tokens
# Essa implementação produz uma representação esparsa das contagens

count_vect = CountVectorizer() # Instancia o algoritmo do count vectorizer
X_train_counts = count_vect.fit_transform(df_train.data)
X_train_counts.shape

(5594, 87600)

In [ ]:
# TF-IDF 
tfidf_transformer = TfidfTransformer() 
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#### **Abordagem 1**

#### **Treinamento do modelo**

In [ ]:
clf = MultinomialNB() # Instancia o algoritmo Naive Bayes para treinamento
clf.fit(X_train_tfidf, df_train.target)

MultinomialNB()

In [ ]:
# Pipeline de machine learning 
clf_1 = Pipeline([
    ('vect', CountVectorizer()),    # Passo 1, aplicar o count vectorizer nos textos 
    ('tfidf', TfidfTransformer()),  # Passo 2, aplicar o TFIDF nos textos
    ('clf', MultinomialNB())])      # Passo 3, aplicar o algoritmo Naive Bayes

In [ ]:
clf_trained = clf_1.fit(df_train.data, df_train.target) # Realiza o treinamento do modelo no pipeline
pred = clf_trained.predict(df_test.data) # Faz predição no dado de teste

In [ ]:
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.7931219774314885


In [ ]:
# gera as métricas de assertividade do modelo
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.83      0.53      0.64       319
           comp.graphics       0.90      0.76      0.82       389
 comp.os.ms-windows.misc       0.88      0.68      0.77       394
comp.sys.ibm.pc.hardware       0.76      0.85      0.80       392
   comp.sys.mac.hardware       0.89      0.83      0.86       385
         rec.motorcycles       0.95      0.97      0.96       398
      rec.sport.baseball       0.93      0.91      0.92       397
        rec.sport.hockey       0.92      0.97      0.95       399
  soc.religion.christian       0.47      0.99      0.64       398
      talk.religion.misc       1.00      0.16      0.28       251

                accuracy                           0.79      3722
               macro avg       0.85      0.76      0.76      3722
            weighted avg       0.85      0.79      0.78      3722



#### **Tuning de parametros**

#### Modelo usando **Naive bayes** com Grid Search

In [ ]:
# Grid Search
# Aqui, estamos criando uma lista de parâmetros para os quais gostaríamos de fazer o ajuste de desempenho.
# Todos os nomes dos parâmetros começam com o nome do classificador (lembre-se do nome arbitrário que demos).
# Por exemplo. vect__ngram_range; aqui estamos dizendo para usar unigramas e bigramas e escolher aquele que é o ideal.

parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [ ]:
gs_clf = GridSearchCV(clf_trained, parameters, n_jobs=-1) # Define o grid search para buscar os melhores parametros 
gs_clf = gs_clf.fit(df_train.data, df_train.target) # treinamento do modelo 

In [ ]:
# Para ver a melhor pontuação média e os parâmetros, execute o seguinte código

print(gs_clf.best_score_)
gs_clf.best_params_

#A saída acima deve ser: A precisão agora aumentou para ~90,6% para o classificador NB (não tão ingênuo mais! 😄)
# e os parâmetros correspondentes são {'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}.

0.9209857063152157


{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [ ]:
pred = gs_clf.predict(df_test.data) # Faz predição no dado de teste

In [ ]:
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8619022031166039


In [ ]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.84      0.85      0.84       319
           comp.graphics       0.83      0.83      0.83       389
 comp.os.ms-windows.misc       0.80      0.67      0.73       394
comp.sys.ibm.pc.hardware       0.72      0.80      0.76       392
   comp.sys.mac.hardware       0.85      0.86      0.86       385
         rec.motorcycles       0.98      0.99      0.98       398
      rec.sport.baseball       0.96      0.93      0.95       397
        rec.sport.hockey       0.95      0.97      0.96       399
  soc.religion.christian       0.85      0.95      0.90       398
      talk.religion.misc       0.82      0.71      0.76       251

                accuracy                           0.86      3722
               macro avg       0.86      0.86      0.86      3722
            weighted avg       0.86      0.86      0.86      3722



# Por fim, podemos observar que dependendo dos parâmetros que utilizamos, podemos obter um bom re